In [14]:
import pandas as pd
import numpy as np

In [17]:
dataset = pd.read_csv(r'../../data/processed/charity_main_cleaned.csv')
ladataset = pd.read_excel(r'../../data/raw/Local_government_finance_data_download.xlsx',sheet_name='Spending power totals')
housing_net_supply_df = pd.read_excel('../../data/raw/Local-authority-housing-supply.xlsx', sheet_name='Net supply')
housing_stock_df = pd.read_excel('../../data/raw/Local-authority-housing-supply.xlsx', sheet_name='Housing stock')
population = pd.read_excel('../../data/raw/population_by_age.xlsx', sheet_name='Age bands')

C:\Users\User\AppData\Local\Temp\ipykernel_22828\925063635.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  dataset = pd.read_csv(r'../../data/processed/charity_main_cleaned.csv')


In [18]:
# Define function to get financial year
def get_financial_year(date):
    if pd.isna(date):
        return np.nan
    return date.year if date.month >= 4 else date.year - 1

# --- ensure dates are in datetime ---
dataset['date_of_registration'] = pd.to_datetime(dataset['date_of_registration'], errors='coerce')
dataset['date_of_removal'] = pd.to_datetime(dataset['date_of_removal'], errors='coerce')

dataset['registration_year'] = dataset['date_of_registration'].dt.to_period('Y')
dataset['removal_year'] = dataset['date_of_removal'].dt.to_period('Y')

dataset['registration_month'] = dataset['date_of_registration'].dt.to_period('M')
dataset['removal_month'] = dataset['date_of_removal'].dt.to_period('M')

dataset['registration_fy'] = dataset['date_of_registration'].apply(get_financial_year)
dataset['removal_fy'] = dataset['date_of_removal'].apply(get_financial_year)

In [3]:
# Step 1: Calculate total 2024 housing stock per council (Local authority only)
stock_2024 = (
    housing_stock_df
    .groupby(['Authority_code', 'Authority_name'], as_index=False)['LA_number']
    .sum()
    .rename(columns={'LA_number': 'Stock_2024'})
)

# Step 2: Prepare Net Supply Data
net = housing_net_supply_df.copy()
net = net[net['Year'].astype(str).str.match(r'^\d{4}/\d{2}$')]  # Keep only year-formatted rows
net['Year'] = net['Year'].str[:4].astype(int)  # Convert '2015/16' → 2015

# Pivot to Local Authority x Year format
net_pivot = net.pivot(index='LA_code', columns='Year', values='Net_additions').fillna(0)
net_pivot = net_pivot[sorted(net_pivot.columns, reverse=True)]  # descending order

# Step 3: Merge with 2024 stock
stock = stock_2024.set_index('Authority_code')
net_pivot = net_pivot.reindex(stock.index)  # align index

# Step 4: Calculate total stock backwards from 2024
stock_by_year = pd.DataFrame(index=net_pivot.index)
stock_by_year[2024] = stock['Stock_2024']

for year in sorted(net_pivot.columns, reverse=True):
    if year < 2024:
        stock_by_year[year] = stock_by_year[year + 1] - net_pivot[year]

# Step 5: Merge back Authority name and reshape
stock_by_year = stock_by_year.merge(stock[['Authority_name']], left_index=True, right_index=True)
stock_by_year_reset = stock_by_year.reset_index().melt(id_vars=['Authority_code', 'Authority_name'], 
                                                       var_name='Year', value_name='Housing stock')

# Final pivot
stock_by_year_pivot = stock_by_year_reset.pivot(index=['Authority_code', 'Authority_name'], 
                                                 columns='Year', values='Housing stock').sort_index(axis=1)

# Reset index, rename, and set it again
stock_by_year_pivot = stock_by_year_pivot.reset_index().rename(columns={'Authority_code': 'ONS code'})
stock_by_year_pivot = stock_by_year_pivot.set_index(['ONS code', 'Authority_name'])


In [4]:
stock_by_year_pivot

,Year,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
ONS code,Authority_name,,,,,,,,,,,,,
E06000001,Hartlepool UA,42292.000000,42371.000000,42420.000000,42761.000000,43254.000000,43404.000000,43634.000000,43970.000000,44190.000000,44316.0,44671.0,45179.0,45764.0
E06000002,Middlesbrough UA,60049.100000,60302.200000,60441.300000,61086.400000,61610.500000,62113.600000,62487.700000,62997.800000,63609.900000,64033.0,64633.0,65236.0,65749.0
E06000003,Redcar and Cleveland UA,62208.900000,62286.800000,62537.700000,63026.600000,63267.500000,63807.400000,64252.300000,64771.200000,65162.100000,65542.0,65986.0,66438.0,66920.0
E06000004,Stockton-on-Tees UA,82692.100000,83292.200000,83634.300000,84059.400000,84407.500000,85315.600000,86069.700000,86851.800000,87847.900000,88448.0,88785.0,89389.0,90055.0
E06000005,Darlington UA,48822.100000,48955.200000,49199.300000,49669.400000,49945.500000,50082.600000,50595.700000,51106.800000,51612.900000,52128.0,52638.0,53149.0,53506.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
E09000029,Sutton,80364.000000,80672.000000,81053.000000,81561.000000,82033.000000,82767.000000,83545.000000,84201.000000,84855.000000,85249.0,85790.0,86267.0,86628.0
E09000030,Tower Hamlets,108766.316335,110239.503409,111411.690483,112839.877557,115746.064631,121085.251705,123600.438778,125636.625852,130712.812926,134473.0,138676.0,141260.0,142331.0
E09000031,Waltham Forest,99003.000000,99691.000000,100300.000000,101191.000000,102384.000000,103637.000000,104569.000000,105402.000000,106537.000000,108172.0,109153.0,110374.0,111318.0


In [5]:
# Filter: Remove NaN and keep only rows where 'Year' matches 'YYYY/YY'
ladataset_filtered = ladataset[
    ladataset['Year'].notna() & ladataset['Year'].str.match(r'^\d{4}/\d{2}$')
]
ladataset_filtered['Year'] = ladataset_filtered['Year'].str[:4].astype(int)

# Pivot
ladataset_pivot = ladataset_filtered.pivot_table(
    index=['ONS code', 'Local authority', 'Measure'],
    columns='Year',
    values='£ millions, cash terms',
    aggfunc='sum'
).reset_index()


C:\Users\User\AppData\Local\Temp\ipykernel_22828\63582683.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ladataset_filtered['Year'] = ladataset_filtered['Year'].str[:4].astype(int)


In [6]:
# Group by 'con_code' only
population_summary = (
    population
    .groupby('con_code', as_index=False)
    .agg({'con_name': 'first', 'con_number': 'sum'})
    .rename(columns={'con_name': 'Local authority', 'con_number': 'Total Population'})
)

# Display the result
print(population_summary.head())



    con_code           Local authority  Total Population
0  E14001063                 Aldershot            119256
1  E14001064       Aldridge-Brownhills             93720
2  E14001065  Altrincham and Sale West            101272
3  E14001066              Amber Valley             90793
4  E14001067   Arundel and South Downs             98053


In [7]:
# Filter for Core Spending Power (CSP)
csp = ladataset_pivot.copy()

# Keep only relevant years
csp = csp.set_index(['ONS code', 'Local authority']).drop(columns='Measure')

# Calculate % change from 2015
csp['financial_distress'] = (csp[2024] - csp[2015]) / csp[2015]

fd = csp[['financial_distress']].reset_index()


In [8]:
fd.columns.name = None

In [9]:
housing_stock_processed = housing_stock_df[housing_stock_df['Tenure'] == 'Local authority'][['Authority_code', 'LA_number']]
fd_new = fd.merge(housing_stock_processed, left_on='ONS code', right_on='Authority_code', how='inner')

In [10]:
fd_new

,ONS code,Local authority,financial_distress,Authority_code,LA_number
0,E06000001,Hartlepool,0.428129,E06000001,358
1,E06000002,Middlesbrough,0.446060,E06000002,36
2,E06000003,Redcar and Cleveland,0.402806,E06000003,0
3,E06000004,Stockton-on-Tees,0.438798,E06000004,0
4,E06000005,Darlington,0.464155,E06000005,5256
...,...,...,...,...,...
291,E09000029,Sutton,0.332970,E09000029,6028
292,E09000030,Tower Hamlets,0.361127,E09000030,11591
293,E09000031,Waltham Forest,0.415258,E09000031,9952
294,E09000032,Wandsworth,0.430561,E09000032,17265


In [11]:
fd_new = fd_new.drop(columns='Authority_code')
fd_new = fd_new.rename(columns={'LA_number': 'LA Housing stock 2024'})

In [28]:
# Step 1: Normalise both variables
fd_new['financial_distress_norm'] = (
    (fd_new['financial_distress'] - fd_new['financial_distress'].min()) /
    (fd_new['financial_distress'].max() - fd_new['financial_distress'].min())
)

fd_new['LA Housing stock 2024_norm'] = (
    (fd_new['LA Housing stock 2024'] - fd_new['LA Housing stock 2024'].min()) /
    (fd_new['LA Housing stock 2024'].max() - fd_new['LA Housing stock 2024'].min())
)

# Step 2: Avoid division by zero and calculate treatment
fd_new['treatment'] = np.where(
    fd_new['LA Housing stock 2024_norm'] != 0,
    fd_new['financial_distress_norm'] / fd_new['LA Housing stock 2024_norm'],
    0
)
fd_new.rename(columns={'Local authority': 'local_authority'}, inplace=True)

In [57]:
fd_new

,ONS code,local_authority,financial_distress,LA Housing stock 2024,financial_distress_norm,LA Housing stock 2024_norm,treatment
0,E06000001,Hartlepool,0.428129,358,0.795629,0.006087,130.707567
1,E06000002,Middlesbrough,0.446060,36,0.825345,0.000612,1348.361066
2,E06000003,Redcar and Cleveland,0.402806,0,0.753664,0.000000,0.000000
3,E06000004,Stockton-on-Tees,0.438798,0,0.813310,0.000000,0.000000
4,E06000005,Darlington,0.464155,5256,0.855332,0.089368,9.570897
...,...,...,...,...,...,...,...
291,E09000029,Sutton,0.332970,6028,0.637931,0.102494,6.224064
292,E09000030,Tower Hamlets,0.361127,11591,0.684594,0.197082,3.473645
293,E09000031,Waltham Forest,0.415258,9952,0.774299,0.169214,4.575849
294,E09000032,Wandsworth,0.430561,17265,0.799660,0.293558,2.724032


In [19]:
# Step 2: Filter valid rows
removed = dataset[dataset['removal_fy'].notnull() & dataset['local_authority'].notnull()].copy()
removed['removal_fy'] = removed['removal_fy'].astype(int)

# Step 3: Group by financial year and local authority
removed_by_fy_la = (
    removed
    .groupby(['local_authority', 'removal_fy'])
    .size()
    .unstack(fill_value=0)
    .sort_index(axis=1)
)

# Step 4: Keep only FY 2015–2024
fy_years = list(range(2015, 2025))
removed_by_fy_la = removed_by_fy_la[removed_by_fy_la.columns.intersection(fy_years)]

# Step 5: Total and sort
removed_by_fy_la['Total'] = removed_by_fy_la.sum(axis=1)
removed_by_fy_la = removed_by_fy_la.sort_values(by='Total', ascending=False)

# Step 6: Preview top 10
print("Removed Charities per Financial Year per Local Authority (FY 2015–2024):")
print(removed_by_fy_la.head(10))

Removed Charities per Financial Year per Local Authority (FY 2015–2024):
removal_fy       2015  2016  2017  2018  2019  2020  2021  2022  2023  2024  \
local_authority                                                               
Westminster       453   523   358   569   771   361   463   506   494   447   
North Yorkshire   381   497   488   558   466   340   462   484   273   521   
Birmingham        393   457   427   523   751   278   508   426   286   381   
Camden            248   351   317   395   564   198   447   273   445   377   
Cornwall          279   355   389   404   365   340   413   320   245   373   
Somerset          402   226   453   324   384   309   333   280   329   404   
Buckinghamshire   268   323   352   389   462   269   360   292   301   295   
Wiltshire         234   302   310   365   431   297   291   338   317   335   
Barnet            325   305   315   295   501   212   248   222   300   318   
County Durham     397   281   346   292   314   266   295 

In [20]:
# Step 1: Define year range
years = list(range(2015, 2025))

# Step 2: Calculate percentage change across years
removed_pct_change = removed_by_fy_la[years].pct_change(axis=1) * 100

# Step 3: Round for readability
removed_pct_change = removed_pct_change.round(2)
removed_pct_change

removal_fy,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
local_authority,,,,,,,,,,
Westminster,NaN,15.45,-31.55,58.94,35.50,-53.18,28.25,9.29,-2.37,-9.51
North Yorkshire,NaN,30.45,-1.81,14.34,-16.49,-27.04,35.88,4.76,-43.60,90.84
Birmingham,NaN,16.28,-6.56,22.48,43.59,-62.98,82.73,-16.14,-32.86,33.22
Camden,NaN,41.53,-9.69,24.61,42.78,-64.89,125.76,-38.93,63.00,-15.28
Cornwall,NaN,27.24,9.58,3.86,-9.65,-6.85,21.47,-22.52,-23.44,52.24
...,...,...,...,...,...,...,...,...,...,...
Shetland Islands,NaN,NaN,NaN,NaN,NaN,NaN,inf,-100.00,NaN,NaN
"Newry, Mourne and Down",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,inf,-100.00
Aberdeenshire,NaN,inf,-100.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [32]:
removed_pct_change

removal_fy,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
local_authority,,,,,,,,,,
Westminster,NaN,15.45,-31.55,58.94,35.50,-53.18,28.25,9.29,-2.37,-9.51
North Yorkshire,NaN,30.45,-1.81,14.34,-16.49,-27.04,35.88,4.76,-43.60,90.84
Birmingham,NaN,16.28,-6.56,22.48,43.59,-62.98,82.73,-16.14,-32.86,33.22
Camden,NaN,41.53,-9.69,24.61,42.78,-64.89,125.76,-38.93,63.00,-15.28
Cornwall,NaN,27.24,9.58,3.86,-9.65,-6.85,21.47,-22.52,-23.44,52.24
...,...,...,...,...,...,...,...,...,...,...
Shetland Islands,NaN,NaN,NaN,NaN,NaN,NaN,inf,-100.00,NaN,NaN
"Newry, Mourne and Down",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,inf,-100.00
Aberdeenshire,NaN,inf,-100.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [40]:
removed_long = (
    removed_pct_change
    .reset_index()
    .melt(id_vars='local_authority', var_name='Year', value_name='removed_rate')
)
removed_long['Year'] = removed_long['Year'].astype(int)


In [41]:
removed_long

,local_authority,Year,removed_rate
0,Westminster,2015,NaN
1,North Yorkshire,2015,NaN
2,Birmingham,2015,NaN
3,Camden,2015,NaN
4,Cornwall,2015,NaN
...,...,...,...
3495,Shetland Islands,2024,NaN
3496,"Newry, Mourne and Down",2024,-100.0
3497,Aberdeenshire,2024,NaN
3498,East Ayrshire,2024,NaN


In [42]:
# Add treatment, post, and controls
panel_df = removed_long.merge(fd_new, on='local_authority', how='left')
panel_df['post'] = (panel_df['Year'] >= 2021).astype(int)
panel_df['treatment_post'] = panel_df['treatment'] * panel_df['post']



In [44]:
panel_df

,local_authority,Year,removed_rate,ONS code,financial_distress,LA Housing stock 2024,financial_distress_norm,LA Housing stock 2024_norm,treatment,post,treatment_post
0,Westminster,2015,NaN,E09000033,0.355025,12207.0,0.674481,0.207556,3.249631,0,0.0
1,North Yorkshire,2015,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN
2,Birmingham,2015,NaN,E08000025,0.466262,58813.0,0.858823,1.000000,0.858823,0,0.0
3,Camden,2015,NaN,E09000007,0.333732,22808.0,0.639194,0.387805,1.648234,0,0.0
4,Cornwall,2015,NaN,E06000052,0.490814,10381.0,0.899511,0.176509,5.096133,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
3495,Shetland Islands,2024,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN
3496,"Newry, Mourne and Down",2024,-100.0,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN
3497,Aberdeenshire,2024,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN
3498,East Ayrshire,2024,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN


In [46]:
panel_df = panel_df[np.isfinite(panel_df['removed_rate'])]
panel_df['removed_rate'].dropna(inplace=True)

C:\Users\User\AppData\Local\Temp\ipykernel_22828\3184187306.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  panel_df['removed_rate'].dropna(inplace=True)


In [48]:
panel_df.drop(columns=['ONS code'], inplace=True)


C:\Users\User\AppData\Local\Temp\ipykernel_22828\2898728315.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  panel_df.drop(columns=['ONS code'], inplace=True)


In [58]:
import statsmodels.formula.api as smf

model = smf.ols('removed_rate ~ post + treatment + C(local_authority)+ treatment:post', data=panel_df).fit()
summary = model.summary()
print(summary.tables[0])  # Overview: R², F-stat, etc.
print(summary.tables[1].as_text())  # Full coefficient table


                            OLS Regression Results                            
Dep. Variable:           removed_rate   R-squared:                       0.095
Model:                            OLS   Adj. R-squared:                 -0.019
Method:                 Least Squares   F-statistic:                    0.8307
Date:                Sun, 29 Jun 2025   Prob (F-statistic):              0.977
Time:                        20:01:37   Log-Likelihood:                -15546.
No. Observations:                2516   AIC:                         3.166e+04
Df Residuals:                    2233   BIC:                         3.331e+04
Df Model:                         282                                         
Covariance Type:            nonrobust                                         
                                                                coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------

In [55]:
panel_df

,local_authority,Year,removed_rate,financial_distress,LA Housing stock 2024,financial_distress_norm,LA Housing stock 2024_norm,treatment,post,treatment_post
350,Westminster,2016,15.45,0.355025,12207.0,0.674481,0.207556,3.249631,0,0.000000
351,North Yorkshire,2016,30.45,NaN,NaN,NaN,NaN,NaN,0,NaN
352,Birmingham,2016,16.28,0.466262,58813.0,0.858823,1.000000,0.858823,0,0.000000
353,Camden,2016,41.53,0.333732,22808.0,0.639194,0.387805,1.648234,0,0.000000
354,Cornwall,2016,27.24,0.490814,10381.0,0.899511,0.176509,5.096133,0,0.000000
...,...,...,...,...,...,...,...,...,...,...
3472,Fife,2024,-100.00,NaN,NaN,NaN,NaN,NaN,1,NaN
3473,Highland,2024,-50.00,NaN,NaN,NaN,NaN,NaN,1,NaN
3478,Dumfries and Galloway,2024,-100.00,NaN,NaN,NaN,NaN,NaN,1,NaN
3485,Isles of Scilly,2024,-100.00,0.438637,115.0,0.813043,0.001955,415.804279,1,415.804279
